# Quadratic Assignment Problem (QAP)

In this notebook, we will see how the Qaudratic Assignment Problem (QAP) can be solved with Quantum Approximate Optimization Algorithm (QAOA) in the qibotn backend on qibo. The QAP formulation has been taken from Qibo Applications and in this notebook implemented using the tensor networks based QAOA simulation. 

In [ ]:
import numpy as np

from qap import qubo_qap, qubo_qap_penalty, qubo_qap_feasibility, qubo_qap_energy, hamiltonian_qap

def load_qap(filename):
    """Load qap problem from a file

    The file format is compatible with the one used in QAPLIB

    """

    with open(filename, 'r') as fh:
        n = int(fh.readline())

        numbers = [float(n) for n in fh.read().split()]

        data = np.asarray(numbers).reshape(2, n, n)
        f = data[1]
        d = data[0]

    i = range(len(f))
    f[i, i] = 0
    d[i, i] = 0

    return f, d

#### Load QAP problem from a file

In [ ]:
F, D = load_qap('tiny04a.dat')
print(f'The QAP instance is:')
print(F)
print(D)

#### Calculate the Penalty

In [ ]:
penalty = qubo_qap_penalty((F, D))
print(f'The penalty is {penalty}')

#### Forumate the QUBO

In [ ]:
linear, quadratic, offset = qubo_qap((F, D), penalty=penalty)
print(f'linear: {linear}')
print()
print(f'quadratic: {quadratic}')
print()
print(f'offset: {offset}\n')

#### Generate a random solution and check its feasibility

rng = np.random.default_rng(seed=1234)
random_solution = {i: rng.integers(2) for i in range(F.size)}
print(f'The random solution is {random_solution}\n')

In [ ]:
feasibility = qubo_qap_feasibility((F, D), random_solution)
print(f'The feasibility of the random solution is {feasibility}\n')

#### Generate a feasible solution and check its feasibility

In [ ]:
feasible_solution = np.zeros(F.shape)
sequence = np.arange(F.shape[0])
np.random.shuffle(sequence)
for i in range(F.shape[0]):
    feasible_solution[i, sequence[i]] = 1
feasible_solution = {k:v for k, v in enumerate(feasible_solution.flatten())}
print(f'The feasible solution is {feasible_solution}\n')

In [ ]:
feasibility = qubo_qap_feasibility((F, D), feasible_solution)
print(f'The feasibility of the feasible solution is {feasibility}\n')

#### Calculate the energy of the feasible solution

In [ ]:
energy = qubo_qap_energy((F,D), feasible_solution)
print(f'The energy of the feasible solution is {energy}')

#### Hamiltonian

In [ ]:
ham = hamiltonian_qap((F, D), dense=False)

##### Solve the Hamiltonian with QAOA

import qibo
from qibo import Circuit

computation_settings = {
    "MPI_enabled": False,
    "MPS_enabled": {
        "qr_method": False,
        "svd_method": {
            "partition": "UV",
            "abs_cutoff": 1e-12,
        },
    },
    "NCCL_enabled": False,
    "expectation_enabled": False,
    "QAOA_execute": {
        "ham_cost": ham,
        "ham_mixer": None,
        "circ_depth": 5,
        "gamma": 2,
        "beta": None 
    }
}

qibo.set_backend(backend="qibotn", platform="cutensornet", runcard=computation_settings)

c = Circuit(nqubits)

# uniform superposition state preparation circuit
for i in range(0, nqubits):
    c.add(gates.H(i))

result = c()

print(result.state())